Q1

In [9]:
import pandas as pd

In [10]:
pip install pyarrow


'2.2.3'

In [12]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [13]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [14]:
num_columns_jan = len(df_jan.columns)

In [15]:
print(f"The number of columns in the January dataset is: {num_columns_jan}")


The number of columns in the January dataset is: 19


Q2

In [7]:
import numpy as np


In [8]:

df_jan['duration'] = (df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']).dt.total_seconds() / 60

std_duration = df_jan['duration'].std()

print(f"The standard deviation of trip durations in January is: {std_duration:.2f} minutes")

The standard deviation of trip durations in January is: 42.59 minutes


  Q3

In [9]:

total_records = len(df_jan)

df_jan_filtered = df_jan[(df_jan['duration'] >= 1) & (df_jan['duration'] <= 60)]

filtered_records = len(df_jan_filtered)

fraction_left = filtered_records / total_records

percentage_left = fraction_left * 100

print(f"Fraction of records left after dropping outliers: {fraction_left:.4f}")
print(f"Percentage of records left after dropping outliers: {percentage_left:.2f}%")

Fraction of records left after dropping outliers: 0.9812
Percentage of records left after dropping outliers: 98.12%


Q4

In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer

# Read the January 2023 data
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

# Select  the pickup and dropoff location IDs
df = df[['PULocationID', 'DOLocationID']]

#  Convert IDs to strings
df['PULocationID'] = df['PULocationID'].astype(str)
df['DOLocationID'] = df['DOLocationID'].astype(str)

# Convert dataframe to list of dictionaries
dict_list = df.to_dict(orient='records')

dv = DictVectorizer()
X = dv.fit_transform(dict_list)

# Get the number of columns  of the feature matrix
num_columns = X.shape[1]

print(f"The dimensionality of the feature matrix is: {num_columns}")


The dimensionality of the feature matrix is: 518


Q5


In [2]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

#Filter for durations between 1 and 60 minutes 
df = df[(df['duration'] >= 1) & (df['duration'] <= 60)].copy()

features = ['PULocationID', 'DOLocationID']
X = df[features]
y = df['duration']

#Convert IDs to strings
X = X.astype(str)

# Convert dataframe to list of dictionaries
dict_list = X.to_dict(orient='records')

dv = DictVectorizer()
X_encoded = dv.fit_transform(dict_list)

# Train a linear regression model
model = LinearRegression()
model.fit(X_encoded, y)

#Make predictions on the training data
y_pred = model.predict(X_encoded)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y, y_pred))

print(f"The RMSE on the training data is: {rmse:.2f}")

The RMSE on the training data is: 7.65


Q6

In [3]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

df_train = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

df_train['duration'] = (df_train['tpep_dropoff_datetime'] - df_train['tpep_pickup_datetime']).dt.total_seconds() / 60

df_train = df_train[(df_train['duration'] >= 1) & (df_train['duration'] <= 60)].copy()

features = ['PULocationID', 'DOLocationID']
X_train = df_train[features].astype(str)
y_train = df_train['duration']

dv = DictVectorizer()
X_train_encoded = dv.fit_transform(X_train.to_dict(orient='records'))

# Train the model
model = LinearRegression()
model.fit(X_train_encoded, y_train)

# Load the validation data (February 2023)
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

# Prepare the target variable for validation data
df_val['duration'] = (df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']).dt.total_seconds() / 60

# Filter validation data for durations between 1 and 60 minutes
df_val = df_val[(df_val['duration'] >= 1) & (df_val['duration'] <= 60)].copy()

# Prepare features and target for validation
X_val = df_val[features].astype(str)
y_val = df_val['duration']

# Transform validation data
X_val_encoded = dv.transform(X_val.to_dict(orient='records'))

# Make predictions on validation data
y_pred = model.predict(X_val_encoded)

#  Calculate RMSE on validation data
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

print(f"The RMSE on the validation data is: {rmse:.2f}")

The RMSE on the validation data is: 7.81
